<a href="https://colab.research.google.com/github/BandaSahithi18/NLP/blob/main/2403A54070_ASS_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK1:LDA - SAMPLE DATA

step1:load the data set using pandas


In [ ]:
import pandas as pd
df=pd.read_excel('/content/LDA-Data.xlsx')
print(df.head())

                             News
0   Virat scored century in match
1            BJP won in elections
2  Bumra took 5 wicket in a match
3  Congress form state government


step2:Text preprocessing


In [ ]:
import re
import pandas as pd

# Load dataset from the available Excel file
df = pd.read_excel('/content/LDA-Data.xlsx')

# Define preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs (http, https, www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove social media mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Remove special characters (keep only alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Normalize whitespace (reduce multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to 'News' column from the loaded Excel data
df['processed_news'] = df['News'].apply(preprocess_text)

# Preview results
print(df[['News', 'processed_news']].head())

                             News                  processed_news
0   Virat scored century in match   virat scored century in match
1            BJP won in elections            bjp won in elections
2  Bumra took 5 wicket in a match  bumra took 5 wicket in a match
3  Congress form state government  congress form state government


word tokenization


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')  # required in newer NLTK versions

df['tokenized_news'] = df['processed_news'].apply(lambda x: word_tokenize(x))
print(df[['processed_news', 'tokenized_news']].head())

                   processed_news                          tokenized_news
0   virat scored century in match     [virat, scored, century, in, match]
1            bjp won in elections               [bjp, won, in, elections]
2  bumra took 5 wicket in a match  [bumra, took, 5, wicket, in, a, match]
3  congress form state government     [congress, form, state, government]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


stop word removal


In [ ]:
import nltk
from nltk.corpus import stopwords

# Ensure the stopwords resource is available
nltk.download('stopwords')

# Define the set of English stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from tokenized_news
df['filtered_news'] = df['tokenized_news'].apply(
    lambda tokens: [w for w in tokens if w.lower() not in stop_words]
)

# Preview results
print(df[['tokenized_news', 'filtered_news']].head())

                           tokenized_news                        filtered_news
0     [virat, scored, century, in, match]      [virat, scored, century, match]
1               [bjp, won, in, elections]                     [bjp, elections]
2  [bumra, took, 5, wicket, in, a, match]      [bumra, took, 5, wicket, match]
3     [congress, form, state, government]  [congress, form, state, government]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


lemmatization

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

# Ensure the WordNet corpus is available
nltk.download('wordnet')
nltk.download('omw-1.4')  # optional, improves lemmatization coverage

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to each token list
df['lemmatized_news'] = df['filtered_news'].apply(
    lambda tokens: [lemmatizer.lemmatize(w) for w in tokens]
)

# Preview results
print(df[['filtered_news', 'lemmatized_news']].head())

                         filtered_news                      lemmatized_news
0      [virat, scored, century, match]      [virat, scored, century, match]
1                     [bjp, elections]                      [bjp, election]
2      [bumra, took, 5, wicket, match]      [bumra, took, 5, wicket, match]
3  [congress, form, state, government]  [congress, form, state, government]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Rejoin

In [ ]:
# Rejoin lemmatized words into a single string
df['clean_news'] = df['lemmatized_news'].apply(lambda tokens: ' '.join(tokens))

# Preview results
print(df[['lemmatized_news', 'clean_news']].head())

                       lemmatized_news                      clean_news
0      [virat, scored, century, match]      virat scored century match
1                      [bjp, election]                    bjp election
2      [bumra, took, 5, wicket, match]       bumra took 5 wicket match
3  [congress, form, state, government]  congress form state government


step5: Apply LDA

In [ ]:
!pip install gensim

from gensim import corpora

# Create dictionary
dictionary = corpora.Dictionary(df['lemmatized_news'])

# Create corpus (bag-of-words representation)
corpus = [dictionary.doc2bow(text) for text in df['lemmatized_news']]

In [ ]:
from gensim.models import LdaModel

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, random_state=42, passes=10)

# Print topics
for idx, topic in lda_model.print_topics(num_words=5):
    print(f"Topic {idx}: {topic}")


Topic 0: 0.199*"bjp" + 0.199*"election" + 0.050*"century" + 0.050*"virat" + 0.050*"scored"
Topic 1: 0.154*"congress" + 0.154*"form" + 0.154*"state" + 0.154*"government" + 0.039*"bjp"
Topic 2: 0.171*"match" + 0.098*"bumra" + 0.098*"5" + 0.098*"wicket" + 0.098*"took"


In [ ]:
from gensim.models import LdaModel

# Train LDA model (example with 3 topics)
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, random_state=42, passes=10)

# Print top words for each topic
num_words = 10  # how many words per topic
topics = lda_model.show_topics(num_topics=3, num_words=num_words, formatted=False)

for topic_num, words in topics:
    print(f"\nTopic {topic_num}:")
    for word, weight in words:
        print(f"  {word} ({weight:.3f})")



Topic 0:
  bjp (0.199)
  election (0.199)
  century (0.050)
  virat (0.050)
  scored (0.050)
  match (0.050)
  took (0.050)
  bumra (0.050)
  5 (0.050)
  wicket (0.050)

Topic 1:
  congress (0.154)
  form (0.154)
  state (0.154)
  government (0.154)
  bjp (0.039)
  virat (0.039)
  election (0.039)
  century (0.039)
  match (0.039)
  scored (0.039)

Topic 2:
  match (0.171)
  bumra (0.098)
  5 (0.098)
  wicket (0.098)
  took (0.098)
  scored (0.097)
  virat (0.097)
  century (0.097)
  election (0.024)
  bjp (0.024)


BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform sentences
bow_matrix = vectorizer.fit_transform(df['clean_news'])

# Convert to array for readability
bow_array = bow_matrix.toarray()

# Show vocabulary
print("Vocabulary:", vectorizer.get_feature_names_out())

# Show Bag-of-Words matrix
print("Bag-of-Words Matrix:\n", bow_array)

Vocabulary: ['bjp' 'bumra' 'century' 'congress' 'election' 'form' 'government' 'match'
 'scored' 'state' 'took' 'virat' 'wicket']
Bag-of-Words Matrix:
 [[0 0 1 0 0 0 0 1 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 1 0 1]
 [0 0 0 1 0 1 1 0 0 1 0 0 0]]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform sentences
bow_matrix = vectorizer.fit_transform(df['clean_news'])

# Convert to DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Add original sentences for reference
bow_df.insert(0, "Sentence", df['clean_news'])

# Display Bag-of-Words DataFrame
print(bow_df.head())

                         Sentence  bjp  bumra  century  congress  election  \
0      virat scored century match    0      0        1         0         0   
1                    bjp election    1      0        0         0         1   
2       bumra took 5 wicket match    0      1        0         0         0   
3  congress form state government    0      0        0         1         0   

   form  government  match  scored  state  took  virat  wicket  
0     0           0      1       1      0     0      1       0  
1     0           0      0       0      0     0      0       0  
2     0           0      1       0      0     1      0       1  
3     1           1      0       0      1     0      0       0  


APPLY NMF

In [ ]:
from sklearn.decomposition import NMF

# Choose number of topics
num_topics = 2

nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(tfidf_matrix)

# Get topic-word matrix
topic_words = nmf_model.components_


In [ ]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()
num_words = 10  # top words per topic

for topic_idx, topic in enumerate(topic_words):
    print(f"\nTopic {topic_idx}:")
    top_indices = topic.argsort()[-num_words:][::-1]
    for i in top_indices:
        print(f"  {feature_names[i]} ({topic[i]:.3f})")



Topic 0:
  match (0.715)
  bumra (0.453)
  wicket (0.453)
  took (0.453)
  virat (0.453)
  scored (0.453)
  century (0.453)
  state (0.000)
  government (0.000)
  election (0.000)

Topic 1:
  election (0.722)
  bjp (0.722)
  form (0.255)
  government (0.255)
  state (0.255)
  congress (0.255)
  took (0.000)
  virat (0.000)
  wicket (0.000)
  scored (0.000)


TASK 2: NMF-BOW KAGGLE DATA

In [ ]:
import pandas as pd
df1=pd.read_csv('/content/arxiv_data.csv')
print(df1.head())

                                              titles  \
0  Survey on Semantic Stereo Matching / Semantic ...   
1  FUTURE-AI: Guiding Principles and Consensus Re...   
2  Enforcing Mutual Consistency of Hard Regions f...   
3  Parameter Decoupling Strategy for Semi-supervi...   
4  Background-Foreground Segmentation for Interio...   

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                         terms  
0           ['cs.CV', 'cs.LG']  
1  ['cs.CV', 'cs.AI', 'cs.LG']  
2           ['cs.CV', 'cs.AI']  
3                    ['cs.CV']  
4           ['cs.CV', 'cs.LG']  


PREPROCESSING

In [ ]:
import re
import pandas as pd

# Load dataset from the available CSV file
df1 = pd.read_csv('/content/arxiv_data.csv')

# Define preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs (http, https, www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove social media mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Remove special characters (keep only alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Normalize whitespace (reduce multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to 'summaries' column from the loaded CSV data
df1['processed_news'] = df1['summaries'].apply(preprocess_text)

# Preview results
print(df1[['summaries', 'processed_news']].head())

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                                      processed_news  
0  stereo matching is one of the widely used tech...  
1  the recent advancements in artificial intellig...  
2  in this paper we proposed a novel mutual consi...  
3  consistency training has proven to be an advan...  
4  to ensure safety in automated driving the corr...  


TOKENIZATION

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')  # required in newer NLTK versions

df1['tokenized_news'] = df1['processed_news'].apply(lambda x: word_tokenize(x))
print(df1[['processed_news', 'tokenized_news']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                      processed_news  \
0  stereo matching is one of the widely used tech...   
1  the recent advancements in artificial intellig...   
2  in this paper we proposed a novel mutual consi...   
3  consistency training has proven to be an advan...   
4  to ensure safety in automated driving the corr...   

                                      tokenized_news  
0  [stereo, matching, is, one, of, the, widely, u...  
1  [the, recent, advancements, in, artificial, in...  
2  [in, this, paper, we, proposed, a, novel, mutu...  
3  [consistency, training, has, proven, to, be, a...  
4  [to, ensure, safety, in, automated, driving, t...  


STOP WORD REMOVAL

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords') # Ensure stopwords is downloaded

# Tokenize 'processed_news'
df1['tokenized_news'] = df1['processed_news'].apply(word_tokenize)

# Define the set of English stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from tokenized_news
df1['filtered_news'] = df1['tokenized_news'].apply(
    lambda tokens: [w for w in tokens if w.lower() not in stop_words]
)

print(df1[['processed_news', 'tokenized_news', 'filtered_news']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                      processed_news  \
0  stereo matching is one of the widely used tech...   
1  the recent advancements in artificial intellig...   
2  in this paper we proposed a novel mutual consi...   
3  consistency training has proven to be an advan...   
4  to ensure safety in automated driving the corr...   

                                      tokenized_news  \
0  [stereo, matching, is, one, of, the, widely, u...   
1  [the, recent, advancements, in, artificial, in...   
2  [in, this, paper, we, proposed, a, novel, mutu...   
3  [consistency, training, has, proven, to, be, a...   
4  [to, ensure, safety, in, automated, driving, t...   

                                       filtered_news  
0  [stereo, matching, one, widely, used, techniqu...  
1  [recent, advancements, artificial, intelligenc...  
2  [paper, proposed, novel, mutual, consistency, ...  
3  [consistency, training, proven, advanced, semi...  
4  [ensure, safety, automated, driving, correct, ..

LEMMATIZATION

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Ensure the WordNet corpus is available
nltk.download('wordnet')
nltk.download('omw-1.4')  # optional, improves lemmatization coverage

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to each token list
df1['lemmatized_news'] = df1['filtered_news'].apply(
    lambda tokens: [lemmatizer.lemmatize(w) for w in tokens]
)

# Preview results
print(df1[['filtered_news', 'lemmatized_news']].head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                       filtered_news  \
0  [stereo, matching, one, widely, used, techniqu...   
1  [recent, advancements, artificial, intelligenc...   
2  [paper, proposed, novel, mutual, consistency, ...   
3  [consistency, training, proven, advanced, semi...   
4  [ensure, safety, automated, driving, correct, ...   

                                     lemmatized_news  
0  [stereo, matching, one, widely, used, techniqu...  
1  [recent, advancement, artificial, intelligence...  
2  [paper, proposed, novel, mutual, consistency, ...  
3  [consistency, training, proven, advanced, semi...  
4  [ensure, safety, automated, driving, correct, ...  


BOW

# Task
Perform LDA topic modeling on the `summaries` column of the `arxiv_data.csv` dataset. This involves cleaning the text by removing URLs, HTML tags, social media mentions, hashtags, emojis, and special characters, converting text to lowercase, normalizing whitespace, then tokenizing, removing stopwords, and lemmatizing. After preprocessing, create a Bag-of-Words representation, prepare the corpus for LDA, apply an LDA model to identify topics, and finally, display the top words for each identified topic.

## Load Text Data

### Subtask:
Load the text data from the provided source (e.g., a CSV file) into a pandas DataFrame. This will typically involve loading the `arxiv_data.csv`.


**Reasoning**:
The subtask explicitly asks to load the text data from `arxiv_data.csv` into a DataFrame named `df1` and display its head. Although this step has appeared before, re-executing it ensures the `df1` DataFrame is correctly initialized as per the task requirements, which is a necessary precursor for subsequent processing steps.



In [15]:
import pandas as pd
df1=pd.read_csv('/content/arxiv_data.csv')
print(df1.head())

                                              titles  \
0  Survey on Semantic Stereo Matching / Semantic ...   
1  FUTURE-AI: Guiding Principles and Consensus Re...   
2  Enforcing Mutual Consistency of Hard Regions f...   
3  Parameter Decoupling Strategy for Semi-supervi...   
4  Background-Foreground Segmentation for Interio...   

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                         terms  
0           ['cs.CV', 'cs.LG']  
1  ['cs.CV', 'cs.AI', 'cs.LG']  
2           ['cs.CV', 'cs.AI']  
3                    ['cs.CV']  
4           ['cs.CV', 'cs.LG']  


## Text Preprocessing - Clean Text

### Subtask:
Apply a series of text cleaning steps to the 'summaries' column, including removing URLs, HTML tags, social media mentions, hashtags, emojis, special characters, converting text to lowercase, and normalizing whitespace. Store the cleaned text in a new column.


**Reasoning**:
The subtask requires applying a series of text cleaning steps to the 'summaries' column. This involves importing the `re` module, defining a `preprocess_text` function to handle various cleaning operations like removing URLs, HTML tags, social media mentions, hashtags, emojis, special characters, converting to lowercase, and normalizing whitespace. Finally, the function will be applied to the 'summaries' column to create a new 'processed_news' column, and the head of both columns will be displayed to verify the cleaning process.



In [16]:
import re

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs (http, https, www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove social media mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Remove special characters (keep only alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Normalize whitespace (reduce multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to 'summaries' column
df1['processed_news'] = df1['summaries'].apply(preprocess_text)

# Preview results
print(df1[['summaries', 'processed_news']].head())

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                                      processed_news  
0  stereo matching is one of the widely used tech...  
1  the recent advancements in artificial intellig...  
2  in this paper we proposed a novel mutual consi...  
3  consistency training has proven to be an advan...  
4  to ensure safety in automated driving the corr...  


## Text Preprocessing - Tokenization

### Subtask:
Tokenize the preprocessed text in the 'processed_news' column into individual words. Store the tokenized words in a new column called 'tokenized_news'.

#### Instructions:
1. Import necessary libraries: `nltk` and `word_tokenize` from `nltk.tokenize`.
2. Download the 'punkt' tokenizer models if not already present.
3. Apply `word_tokenize` to the 'processed_news' column and store the result in a new column named 'tokenized_news'.
4. Display the first five rows of the 'processed_news' and 'tokenized_news' columns to verify the tokenization.

**Reasoning**:
The subtask requires tokenizing the preprocessed text. This involves importing `nltk` and `word_tokenize`, ensuring the 'punkt' tokenizer models are downloaded, applying `word_tokenize` to the `processed_news` column, and storing the result in a new `tokenized_news` column. Finally, the head of both columns will be displayed to verify the tokenization.



In [17]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')  # required in newer NLTK versions

df1['tokenized_news'] = df1['processed_news'].apply(lambda x: word_tokenize(x))
print(df1[['processed_news', 'tokenized_news']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                      processed_news  \
0  stereo matching is one of the widely used tech...   
1  the recent advancements in artificial intellig...   
2  in this paper we proposed a novel mutual consi...   
3  consistency training has proven to be an advan...   
4  to ensure safety in automated driving the corr...   

                                      tokenized_news  
0  [stereo, matching, is, one, of, the, widely, u...  
1  [the, recent, advancements, in, artificial, in...  
2  [in, this, paper, we, proposed, a, novel, mutu...  
3  [consistency, training, has, proven, to, be, a...  
4  [to, ensure, safety, in, automated, driving, t...  


**Reasoning**:
The previous step successfully tokenized the text. The next logical step in text preprocessing for topic modeling is to remove common stopwords, as these words often do not contribute meaningful information to topic identification. This will be followed by a reasoning block and the code block for stop word removal.



**Reasoning**:
The previous code block failed because it contained markdown instructions within a Python code cell. The intent was to perform stop word removal, so this code block will provide the correct Python implementation for that task, assuming the markdown instructions will be placed in a separate markdown cell.



In [19]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords') # Ensure stopwords is downloaded

# Define the set of English stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from tokenized_news
df1['filtered_news'] = df1['tokenized_news'].apply(
    lambda tokens: [w for w in tokens if w.lower() not in stop_words]
)

print(df1[['tokenized_news', 'filtered_news']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                      tokenized_news  \
0  [stereo, matching, is, one, of, the, widely, u...   
1  [the, recent, advancements, in, artificial, in...   
2  [in, this, paper, we, proposed, a, novel, mutu...   
3  [consistency, training, has, proven, to, be, a...   
4  [to, ensure, safety, in, automated, driving, t...   

                                       filtered_news  
0  [stereo, matching, one, widely, used, techniqu...  
1  [recent, advancements, artificial, intelligenc...  
2  [paper, proposed, novel, mutual, consistency, ...  
3  [consistency, training, proven, advanced, semi...  
4  [ensure, safety, automated, driving, correct, ...  


**Reasoning**:
The previous step successfully removed stopwords. The next logical step in text preprocessing for topic modeling is lemmatization, which reduces words to their base or dictionary form, further normalizing the text. This will be followed by a reasoning block and the code block for lemmatization.



In [21]:
import nltk
from nltk.stem import WordNetLemmatizer

# Ensure the WordNet corpus is available
nltk.download('wordnet')
nltk.download('omw-1.4')  # optional, improves lemmatization coverage

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to each token list
df1['lemmatized_news'] = df1['filtered_news'].apply(
    lambda tokens: [lemmatizer.lemmatize(w) for w in tokens]
)

# Preview results
print(df1[['filtered_news', 'lemmatized_news']].head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                       filtered_news  \
0  [stereo, matching, one, widely, used, techniqu...   
1  [recent, advancements, artificial, intelligenc...   
2  [paper, proposed, novel, mutual, consistency, ...   
3  [consistency, training, proven, advanced, semi...   
4  [ensure, safety, automated, driving, correct, ...   

                                     lemmatized_news  
0  [stereo, matching, one, widely, used, techniqu...  
1  [recent, advancement, artificial, intelligence...  
2  [paper, proposed, novel, mutual, consistency, ...  
3  [consistency, training, proven, advanced, semi...  
4  [ensure, safety, automated, driving, correct, ...  


**Reasoning**:
The previous step successfully lemmatized the text. The next logical step for preparing text for LDA is to rejoin the lemmatized tokens into a single string for each document. This aggregated string will serve as the clean text input for further vectorization or model training.



**Reasoning**:
The previous code block failed with a `SyntaxError` because it contained markdown instructions within a Python code cell. The intent was to rejoin the lemmatized tokens into a single string for each document, so this code block will provide the correct Python implementation for that task, separating the instructional text from the Python code.



In [23]:
df1['clean_news'] = df1['lemmatized_news'].apply(lambda tokens: ' '.join(tokens))

# Preview results
print(df1[['lemmatized_news', 'clean_news']].head())

                                     lemmatized_news  \
0  [stereo, matching, one, widely, used, techniqu...   
1  [recent, advancement, artificial, intelligence...   
2  [paper, proposed, novel, mutual, consistency, ...   
3  [consistency, training, proven, advanced, semi...   
4  [ensure, safety, automated, driving, correct, ...   

                                          clean_news  
0  stereo matching one widely used technique infe...  
1  recent advancement artificial intelligence ai ...  
2  paper proposed novel mutual consistency networ...  
3  consistency training proven advanced semisuper...  
4  ensure safety automated driving correct percep...  


## Bag-of-Words Representation

### Subtask:
Create a Bag-of-Words (BOW) representation from the `clean_news` column using `CountVectorizer`. Display the vocabulary and the BOW matrix.

#### Instructions:
1. Import `CountVectorizer` from `sklearn.feature_extraction.text`.
2. Initialize `CountVectorizer`.
3. Fit and transform the `clean_news` column to get the BOW matrix.
4. Convert the BOW matrix to an array for easier viewing.
5. Print the vocabulary (feature names) obtained from the vectorizer.
6. Print the generated Bag-of-Words matrix.

## Bag-of-Words Representation

### Subtask:
Create a Bag-of-Words (BOW) representation from the `clean_news` column using `CountVectorizer`. Display the vocabulary and the BOW matrix.

#### Instructions:
1. Import `CountVectorizer` from `sklearn.feature_extraction.text`.
2. Initialize `CountVectorizer`.
3. Fit and transform the `clean_news` column to get the BOW matrix.
4. Convert the BOW matrix to an array for easier viewing.
5. Print the vocabulary (feature names) obtained from the vectorizer.
6. Print the generated Bag-of-Words matrix.

**Reasoning**:
The previous markdown block outlined the steps to create a Bag-of-Words representation. This code block will implement those steps by importing `CountVectorizer`, initializing it, fitting and transforming the `clean_news` column, and then printing the vocabulary and the resulting BOW matrix.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform sentences
bow_matrix = vectorizer.fit_transform(df1['clean_news'])

# Convert to array for readability
bow_array = bow_matrix.toarray()

# Show vocabulary
print("Vocabulary:", vectorizer.get_feature_names_out())

# Show Bag-of-Words matrix
print("Bag-of-Words Matrix:\n", bow_array)

**Reasoning**:
The previous markdown block outlined the steps to create a Bag-of-Words representation. This code block will implement those steps by importing `CountVectorizer`, initializing it, fitting and transforming the `clean_news` column, and then printing the vocabulary and the resulting BOW matrix.



In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

# 1. Load data to ensure df1 is defined
df1 = pd.read_csv('/content/arxiv_data.csv')

# 2. Define and apply preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs (http, https, www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove social media mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Remove special characters (keep only alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Normalize whitespace (reduce multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text).strip()

    return text
df1['processed_news'] = df1['summaries'].apply(preprocess_text)

# 3. Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')
df1['tokenized_news'] = df1['processed_news'].apply(lambda x: word_tokenize(x))

# 4. Stop word removal
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df1['filtered_news'] = df1['tokenized_news'].apply(
    lambda tokens: [w for w in tokens if w.lower() not in stop_words]
)

# 5. Lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
df1['lemmatized_news'] = df1['filtered_news'].apply(
    lambda tokens: [lemmatizer.lemmatize(w) for w in tokens]
)

# 6. Rejoin clean text
df1['clean_news'] = df1['lemmatized_news'].apply(lambda tokens: ' '.join(tokens))

# 7. Create Bag-of-Words representation
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(df1['clean_news'])
bow_array = bow_matrix.toarray()

# Show vocabulary
print("Vocabulary:", vectorizer.get_feature_names_out())

# Show Bag-of-Words matrix
print("Bag-of-Words Matrix:\n", bow_array)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**Reasoning**:
The previous code block failed because the `df1` DataFrame and its processed column `clean_news` were not defined in the current execution context. This fix re-initializes `df1` by loading the data and re-applies all necessary preprocessing steps (cleaning, tokenization, stop word removal, lemmatization, and rejoining) to ensure `clean_news` is available before `CountVectorizer` is applied, thus resolving the `NameError`.



In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

# 1. Load data to ensure df1 is defined
df1 = pd.read_csv('/content/arxiv_data.csv')

# 2. Define and apply preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs (http, https, www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove social media mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Remove special characters (keep only alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Normalize whitespace (reduce multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text).strip()

    return text
df1['processed_news'] = df1['summaries'].apply(preprocess_text)

# 3. Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')
df1['tokenized_news'] = df1['processed_news'].apply(lambda x: word_tokenize(x))

# 4. Stop word removal
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df1['filtered_news'] = df1['tokenized_news'].apply(
    lambda tokens: [w for w in tokens if w.lower() not in stop_words]
)

# 5. Lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
df1['lemmatized_news'] = df1['filtered_news'].apply(
    lambda tokens: [lemmatizer.lemmatize(w) for w in tokens]
)

# 6. Rejoin clean text
df1['clean_news'] = df1['lemmatized_news'].apply(lambda tokens: ' '.join(tokens))

# 7. Create Bag-of-Words representation
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(df1['clean_news'])
bow_array = bow_matrix.toarray()

# Show vocabulary
print("Vocabulary:", vectorizer.get_feature_names_out())

# Show Bag-of-Words matrix
print("Bag-of-Words Matrix:\n", bow_array)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**Reasoning**:
The previous code block successfully created the Bag-of-Words representation. The next step, as per the overall task, is to prepare the corpus for LDA using `gensim.corpora.Dictionary` and `doc2bow` from the `lemmatized_news` column.



## Prepare Corpus for LDA

### Subtask:
Install the `gensim` library and then create a Gensim dictionary and corpus (bag-of-words format) from the lemmatized and tokenized text.


**Reasoning**:
The previous attempt to import `gensim` failed because the library was not installed. This step will install the `gensim` library using `!pip install gensim` so that it can be imported and used in subsequent steps.



In [3]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 38.0 MB/s eta 0:00:00


**Reasoning**:
Now that `gensim` is installed, I will import `gensim.corpora`, create a dictionary from the `lemmatized_news` column of `df1`, then create the corpus using `doc2bow`, and finally print verification details as per the instructions.



**Reasoning**:
The previous code block failed because `df1` was not defined in the current execution context. This fix re-initializes `df1` and re-applies all necessary preprocessing steps to create the `lemmatized_news` column before proceeding with the `gensim` dictionary and corpus creation.



In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Example dataset (you can replace this with your Excel file)
data = {
    "Sentence": [
        "The cat sat on the mat",
        "She loves music",
        "Apples are red",
        "The car engine broke down"
    ]
}
df = pd.DataFrame(data)

# Extract sentences
sentences = df["Sentence"].astype(str).tolist()

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform sentences
tfidf_matrix = vectorizer.fit_transform(sentences)

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Add original sentences for reference
tfidf_df.insert(0, "Sentence", sentences)

# Preview
print(tfidf_df)


                    Sentence    apples    broke      car      cat   engine  \
0     The cat sat on the mat  0.000000  0.00000  0.00000  0.57735  0.00000   
1            She loves music  0.000000  0.00000  0.00000  0.00000  0.00000   
2             Apples are red  0.707107  0.00000  0.00000  0.00000  0.00000   
3  The car engine broke down  0.000000  0.57735  0.57735  0.00000  0.57735   

      loves      mat     music       red      sat  
0  0.000000  0.57735  0.000000  0.000000  0.57735  
1  0.707107  0.00000  0.707107  0.000000  0.00000  
2  0.000000  0.00000  0.000000  0.707107  0.00000  
3  0.000000  0.00000  0.000000  0.000000  0.00000  


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Example dataset (replace with your Excel file if needed)
data = {
    "Sentence": [
        "The cat sat on the mat",
        "She loves music",
        "Apples are red",
        "The car engine broke down"
    ]
}
df = pd.DataFrame(data)

# Extract sentences
sentences = df["Sentence"].astype(str).tolist()

# Initialize CountVectorizer (BoW)
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform sentences
bow_matrix = vectorizer.fit_transform(sentences)

# Convert to DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Add original sentences for reference
bow_df.insert(0, "Sentence", sentences)

# Preview
print(bow_df)


                    Sentence  apples  broke  car  cat  engine  loves  mat  \
0     The cat sat on the mat       0      0    0    1       0      0    1   
1            She loves music       0      0    0    0       0      1    0   
2             Apples are red       1      0    0    0       0      0    0   
3  The car engine broke down       0      1    1    0       1      0    0   

   music  red  sat  
0      0    0    1  
1      1    0    0  
2      0    1    0  
3      0    0    0  


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Example dataset (replace with your Excel file)
data = {
    "Sentence": [
        "The cat sat on the mat",
        "She loves music",
        "Apples are red",
        "The car engine broke down"
    ]
}
df = pd.DataFrame(data)

# Step 1: Prepare corpus
sentences = df["Sentence"].astype(str).tolist()

# Step 2: TF-IDF representation
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(sentences)

# Step 3: Apply NMF
num_topics = 2   # you can adjust this
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(tfidf_matrix)

# Step 4: Identify words for each topic
feature_names = vectorizer.get_feature_names_out()
num_words = 5

for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"\nTopic {topic_idx}:")
    top_indices = topic.argsort()[-num_words:][::-1]
    for i in top_indices:
        print(f"  {feature_names[i]} ({topic[i]:.3f})")

# Step 5: Assign topics to sentences
doc_topic_matrix = nmf_model.transform(tfidf_matrix)
df["Topic"] = doc_topic_matrix.argmax(axis=1)

print("\nSentence-to-topic mapping:")
print(df[["Sentence", "Topic"]])



Topic 0:
  car (0.683)
  engine (0.683)
  broke (0.683)
  music (0.205)
  loves (0.205)

Topic 1:
  sat (0.671)
  mat (0.671)
  cat (0.671)
  loves (0.000)
  music (0.000)

Sentence-to-topic mapping:
                    Sentence  Topic
0     The cat sat on the mat      1
1            She loves music      0
2             Apples are red      0
3  The car engine broke down      0
